In [9]:
import cv2
from pyzbar import pyzbar
import pyrealsense2 as rs
import numpy as np

take_measurement_flag=True #make this flag True when the arm reaches the required waypoint
landmark_measurements=[]

def QRcode_center(frame):
    barcodes = pyzbar.decode(frame)
    centerOfQRs=[]
    landmark_id='NIL'
    corners=[]
    for barcode in barcodes:
        landmark_id=barcode.data.decode()
        x, y , w, h = barcode.rect
        cv2.rectangle(frame, (x, y),(x+w, y+h), (0, 255, 0), 2)
        center=(int(x+w/2),int(y+h/2))
        centerOfQRs.append([landmark_id,center[0],center[1]])
        
        left = barcode.rect[0]
        top = barcode.rect[1]
        width = barcode.rect[2]
        height = barcode.rect[3]


        # get the rectangular contour corner coordinates
        top_left = [top,left]
       # print(f'top_left={top_left}')
        top_right = [top,left+width]
       # print(f'top_right={top_right}')
        bottom_left = [top-height,left]
       # print(f'bottom_left={bottom_left}')
        bottom_right = [top-height,left+width]
       # print(f'bottom_right={bottom_right}')
        
        corners=[top_left,top_right,bottom_left,bottom_right]
    return frame,centerOfQRs,corners

def intrinsic_mat(intr):
    width=intr.width
    h=intr.height
    ppx=intr.ppx
    ppy=intr.ppy
    fx=intr.fx
    fy=intr.fy 

        

    intr_mat=np.vstack(([fx,0,ppx],[0,fy,ppy],[0,0,1]))
    
    return intr_mat
    
    
    
    

def landmark_measurement(image_coords,depth_image,intr):
    
    width=intr.width
    h=intr.height
    ppx=intr.ppx
    ppy=intr.ppy
    fx=intr.fx
    fy=intr.fy 
    #landmark_id=image_coords[0]
    [ix,iy]=image_coords[:]
    Zc=depth_image.get_distance(ix, iy)
        
    
     #homography
    P_transf=np.vstack(([Zc/(fx),0,-Zc*ppx/fx],[0,Zc/fy,-Zc*ppy/fy],[0,0,Zc]))
    I=np.transpose([ix,iy,1])
    X=P_transf@I #transform x image_coordinates=camera coordinates
    X[0]=round(X[0]*100,2)
    X[1]=round(X[1]*100,2)
    X[2]=round(X[2]*100,2)
    measurement=[X[0],X[1],X[2]]
    

    return measurement

In [12]:
pipe = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
profile = pipe.start(config)

depth_sensor = profile.get_device().first_depth_sensor()
depth_sensor.set_option(rs.option.visual_preset, 3) # Low ambient light

landmark_measurements=[]
while take_measurement_flag:
    frameset = pipe.wait_for_frames()
    color_frame = frameset.get_color_frame()
    depth=frameset.get_depth_frame()
    color = np.asanyarray(color_frame.get_data())
    frame,centerOfQRs, corners = QRcode_center(color)
    prof = profile.get_stream(rs.stream.depth)
    intr = prof.as_video_stream_profile().get_intrinsics()
    
    intr_mat=intrinsic_mat(intr)
    
    try:
        landmark_measurements=[]
        
        for corner in corners:
            print('corner=',corner)
            [x,y]=corner
            landmark_coordinates_current=landmark_measurement(corner,depth,intr)
            #print('returns=',landmark_coordinates_current)
            landmark_measurements.append(landmark_coordinates_current)
            font = cv2.FONT_HERSHEY_DUPLEX
            print('for lm=',centerOfQRs[0][0],corners)
            cv2.putText(frame, 'landmark measurement='+str(corners), (x, y), font, 0.5, (0,0,0), 2)
        print('lm=',landmark_measurements,'corners=',corners)
        landmark_measurements=np.array(landmark_measurements,dtype=np.float32)
        corners=np.array(corners,dtype=np.float32)
        print(landmark_measurements.shape)
        print(corners.shape)
        
        
        retval, rvec, tvec,_=cv2.solvePnPRansac(landmark_measurements,corners,np.array(intr_mat),None)
        #rvec is the vector with the orientation angles 
        [rot,jacob]=cv2.Rodrigues(rvec)
        print('rx=',rot)
        
       
    except:
        (x,y)=(0,0)
        
    #image = cv2.circle(frame, center, radius=1, color=(0, 0, 255), thickness=5)
    #font = cv2.FONT_HERSHEY_DUPLEX
    #cv2.putText(frame, 'center='+str(center), (x + 6, y - 6), font, 1.0, (0,0,0), 1)
    cv2.imshow('Center of the QR Code', frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break
    #set take_measurement_flag to False here
cv2.destroyAllWindows()

lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners= []
(0,)
(0,)
lm= [] corners

In [ ]:

corners=np.array([290.0, 279.0, 290.0, 428.0, 135.0, 279.0, 135.0, 428.0],dtype=np.float32)
corners=corners.reshape(4,2)
lm= np.array([-1.53, 2.35, 21.1,-1.37, 9.5, 19.0,-9.97, 2.32, 20.9,-8.63, 9.05, 18.1],dtype=np.float32)
lm=lm.reshape(4,3)

dist=np.array([])
retval, rvec, tvec,_=cv2.solvePnPRansac(lm,corners,np.array(intr_mat),None)
[rot,jacob]=cv2.Rodrigues(rvec)
print('rx=',rot,'tx',tvec)


rx= [[ 0.99968771  0.01133921 -0.02226899]
 [-0.00478182  0.96145936  0.2749055 ]
 [ 0.02452794 -0.27471316  0.96121333]] tx [[ 0.4316799 ]
 [-5.69882769]
 [ 1.70746307]]


In [ ]:
lm= np.array([[-8.97, -15.03, 55.7], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]) 
lm=lm.reshape(4,3)
corners=np.array([[256, 133], [256, 280], [113, 133], [113, 280]])
corners=corners.reshape(4,2)
retval, rvec, tvec,_=cv2.solvePnPRansac(lm,corners,np.array(intr_mat),None)
[rot,jacob]=cv2.Rodrigues(rvec)
print('rx=',rot,'tx',tvec)

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\calib3d\src\solvepnp.cpp:241: error: (-215:Assertion failed) npoints >= 4 && npoints == std::max(ipoints.checkVector(2, CV_32F), ipoints.checkVector(2, CV_64F)) in function 'cv::solvePnPRansac'


In [8]:
pi_inv=np.array([[-8.31453656e-01 ,4.20094212e-01 ,3.63600977e-01,-2.72348518e+01],
 [ 3.89441076e-01 ,9.07421557e-01,-1.57866291e-01 ,6.46672105e+00],
 [-3.96258080e-01 ,1.03426512e-02 ,-9.18080913e-01  ,3.41312741e+01],
 [ 0.00000000e+00  ,0.00000000e+00  ,0.00000000e+00  ,1.00000000e+00]])